Перед запуском файл  "Log" нужно переименовать в "Log.csv"!

импортируем модули

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np

откроем логи

In [2]:
data = pd.read_csv('Log.csv', sep='\t', encoding='utf-8')

посмотрим на данные

In [3]:
data.head()

,timestamp,datetime,device,numdoc,region,request,urls
0,1520974800,2018-03-14 00:00,desktop,317731505,6,https://yandex.ru/search/?text=порно видео сек...,http://porno-sex-video.com/eroticheskie-filmy-...
1,1520974800,2018-03-14 00:00,desktop,277053268,191,https://yandex.ru/search/?text=sad story скача...,http://mp3party.net/music/8507899;http://muz-c...
2,1520974800,2018-03-14 00:00,touch,19987730,2,https://yandex.ru/search/?text=ольгино баня&fl...,http://www.bani.spb.ru/olgino.html;https://sau...
3,1520974800,2018-03-14 00:00,touch,196215754,26955,https://yandex.ru/search/?text=фирменный магаз...,http://www.marko.by/actions/;https://marko.vit...
4,1520974800,2018-03-14 00:00,desktop,3909882795,101704,https://yandex.ru/search/?text=порно&rl=313,http://porno365.xxx/;https://prostoporno.sex/c...


посмотрим количество запросов в файле

In [4]:
len(data)

1333503

выделим текстовую часть запросов

In [5]:
request = data['request']
requestList=[]
for i in range(len(request)):
    requestList.append(request[i].replace('https://yandex.ru/search/?text=', '').split('&')[0])

для удобства запишем в файл

In [6]:
f = open('1.txt', 'w')
for i in range(len(requestList)):    
    f.write(requestList[i].encode('utf8')+'\n')
f.close()

In [8]:
#подготовим данные к работе
dat = open('1.txt')
data = dat.readlines()
a, b= [], []
#создадим список запросов
for i in range(len(data)):
    a.append(data[i].strip().lower())
    
#создадим список списков запросов
import re
for j in range(len(a)):
    b.append(re.split(' ', a[j]))

Так как объем данных очень большой, не получится проанализировать все строки. Отфильтруем логи, в которых отсутсвуют ключевые слова, связанные с телевидением.  Поиск таких слов был осуществлен с помощью сервиса по подбору слов Яндекса. были введены запросы "телевидение" и "тв".

создание списка ключевых слов для фильтрации

In [9]:
#filtrWords = '1канал антенна кабельное кабельный tv канал матч муз первый передач передача передачи подключить приставка программа прямая прямое прямом прямой рен ресивер россия1 ростелеком спутниковое спутниковый тв телевидение телевидения телевизионный телевизор телеканал триколор цифровое цифровой эфир эфирный нтв тв-3'
filtrWords = '1канал антенна кабельное кабельный tv канал матч муз первый передач передача передачи подключить приставка программа прямая прямое прямом прямой рен ресивер россия1 ростелеком спутниковое спутниковый тв телевидение телевидения телевизионный телевизор телеканал триколор цифровое цифровой эфир эфирный нтв тв-3'
filtr = []
filtr.append(re.split(' ', filtrWords))

создадим множество из слов-фильтров

In [10]:
words = set()
for i in range(len(filtr)):
    for j in range(len(filtr[i])):
        words.add(filtr[i][j])

добавляем подходящие запросы в новый список

In [11]:
%%time 
newList=[]
for i in range(len(b)):
    for j in range(len(b[i])):
        if b[i][j] in words:
            newList.append(a[i])

Wall time: 1.47 s


оценим количество подходящих запросов

In [12]:
len(newList)

18842

оцением их долю в общем потоке

In [13]:
len(newList)/float(len(data))

0.014129701995421082

запишем подходящие запросы в файл

In [14]:
f = open('2.txt', 'w') 
for i in range(len(newList)):    
    f.write(newList[i]+'\n')
f.close()

Я бы разделил запросы на следующие группы:<p>
1) запрос на просмотр телевизора онлайн (аналог кнопки "ВКЛ" на пульте) - simpLookWords<p>
2) запрос на просмотр конкретного канала или программы телепередач(человек знает, что хочет смотреть) - lookChanWords<p>
3) запрос на тв-оборудование или провайдера (человек нуждается в услуге или товаре) - supportWords

In [15]:
simpLookWords = 'tv прямая прямое прямом прямой эфир'
lookChanWords = '1канал канал матч муз первый передач передача передачи рен россия1 телеканал триколор нтв тв-3'
supportWords = 'антенна кабельное кабельный подключить приставка программа ресивер ростелеком спутниковое спутниковый телевидение телевидения телевизионный телевизор цифровое цифровой эфирный'
#создадим списки
simpLook, lookChan, support = [], [], []
simpLook.append(re.split(' ', simpLookWords))
lookChan.append(re.split(' ', lookChanWords))
support.append(re.split(' ', supportWords))

In [16]:
#создадим множества
simpLookWords = set()
for i in range(len(simpLook)):
    for j in range(len(simpLook[i])):
        simpLookWords.add(simpLook[i][j])
lookChanWords = set()
for i in range(len(lookChan)):
    for j in range(len(lookChan[i])):
        lookChanWords.add(lookChan[i][j])
supportWords = set()
for i in range(len(support)):
    for j in range(len(support[i])):
        supportWords.add(support[i][j])

In [17]:
#готовим данные к работе
dat2 = open('2.txt')
data2 = dat2.readlines()
a2, b2= [], []
#создадим список запросов
for i in range(len(data2)):
    a2.append(data2[i].strip().lower())
    
#создадим список списков запросов
import re
for j in range(len(a2)):
    b2.append(re.split(' ', a2[j]))

In [18]:
#посчитаем вхождения
l_1 = 0
l_2 = 0
l_3 = 0
for i in range(len(b2)):
    for j in range(len(b2[i])):
        if b2[i][j] in simpLookWords:
            l_1 += 1
    for j in range(len(b2[i])):
        if b2[i][j] in lookChanWords:
            l_2 += 1
    for j in range(len(b2[i])):
        if b2[i][j] in supportWords:
            l_3 += 1

Получаем количество вхождений. Это не очень точный результат, так как в одном запросе могут быть ключевые слова из разных групп. Ключевые слова распределял по группам экспериментально.

In [19]:
print(l_1, l_2, l_3)

(11351, 10889, 6201)


Соотношение групп запросов: 1 : 0.959 : 0.546

Более точные результаты можно получить, анализируя косинусные расстояния между векторами вхождения слов в запрос. <p> Так как размер исходной матрицы очень большой, мощностей машины не хватает для их обработки (в один вектор обрабатывается больше 5 минут, с учетом их количества 1.3 млн дальнейший расчет нецелесообразен).

Вот как можно применить метод косинусных расстояний.

In [20]:
#Функция для создания словаря - мешка слов
def createBagWords(listAllwords):
    #создаем множество всех слов
    words = set()
    for i in range(len(listAllwords)):
        for j in range(len(listAllwords[i])):
            words.add(listAllwords[i][j])
    #создаем словарь, где ключ - порядковый номер, значение - слово из мешка
    keys = [i for i in range(len(words))]
    words = dict(zip(words, keys))
    return words

#функция на вход получает запрос, на выходе получаем вектор количества вхождений слов из мешка в наш запрос
def createVector(request):
    vector = np.zeros(len(words))
    for i in range(len(words)):
        if words.keys()[i] in request:
            vector[i]+=1
    return vector

Попробуем проанализировать таким методом уже отобранные логи (newList).

In [21]:
words = createBagWords(newList)

сфомируем матрицы из векторов вхождений

In [22]:
%%time
matrix = np.zeros((len(newList), len(words)))

for i in range(len(newList)):
    for j in range(len(words)):
        if words.keys()[j] in newList[i]:
            matrix[i][j] += 1

Wall time: 3.66 s


создадим вектор, с которым будем сравнивать вектора

In [23]:
normal = []
for i in range(len(filtrWords)):
    normal.append(filtrWords[i].strip().lower())
normal = createVector(normal)

In [24]:
from scipy.spatial.distance import cosine

Рассмотрим несколько пар кос. расстояние - поисковый запрос.

In [25]:
for i in range(100):
    print cosine(matrix[i], normal), newList[i]

0.213281606163 смотреть онлайн матч футбол 1 прямая трансляция
0.213281606163 смотреть онлайн матч футбол 1 прямая трансляция
0.326366930291 рома шахтер прямая трансляция смотреть 13 марта
0.32056138714 пакет ночной триколор тв на 2018 год цена
0.32056138714 пакет ночной триколор тв на 2018 год цена
0.455418851363 программа передач
0.455418851363 программа передач
0.271383260595 второй канал онлайн прямой эфир россия 1
0.271383260595 второй канал онлайн прямой эфир россия 1
0.271383260595 второй канал онлайн прямой эфир россия 1
0.34807975948 матч футбол 1 iptv ссылка
0.404145976932 россия 1 прямой эфир
0.404145976932 россия 1 прямой эфир
0.429218207015 познавательное тв
0.911438511446 pro tv md gindeste liber
0.429218207015 первый канал
0.429218207015 первый канал
0.251518811435 не включается задняя передача октавия тур
0.534621079004 телеканал шалун
0.399754952001 режим спутника на смарт тв
0.511906469908 тв онлайн
0.451204527544 kaban tv online смотреть бесплатно
0.575562656186 матч

Как видно, величина кос. расстояния уже не может дать нам никакой информации.<p> Это связано с тем, что мы отбирали эти запросы по фильтру, который использовали в нормали.<p>
Елси бы размер исходного файла был меньше, векторный анализ был бы успешен.